In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Step 1: Load the dataset
url = "https://storage.googleapis.com/download.tensorflow.org/data/jena_climate_2009_2016.csv.zip"
csv_path = tf.keras.utils.get_file(origin=url, fname="jena_climate.csv", extract=True)
df = pd.read_csv(csv_path)

# Step 2: Select relevant columns (temperature in °C)
data = df[['T (degC)']].values  # we’ll forecast temperature
data = data.astype('float32')

# Normalize the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Step 3: Create sequences
def create_sequences(data, time_steps=144):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
        y.append(data[i+time_steps])
    return np.array(X), np.array(y)

# Each time_step = 144 → 1 day (since data recorded every 10 mins → 144 points/day)
time_steps = 144
X, y = create_sequences(data_scaled, time_steps)

# Step 4: Train-test split
train_size = int(0.8 * len(X))
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

# Step 5: Build LSTM model
model = Sequential([
    LSTM(64, input_shape=(time_steps, 1)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.summary()

# Step 6: Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Step 7: Predict and plot
y_pred = model.predict(X_test)
y_pred_rescaled = scaler.inverse_transform(y_pred)
y_test_rescaled = scaler.inverse_transform(y_test)

plt.figure(figsize=(10,5))
plt.plot(y_test_rescaled[:500], label="Actual")
plt.plot(y_pred_rescaled[:500], label="Predicted")
plt.legend()
plt.title("Temperature Forecasting with LSTM (Jena Climate Data)")
plt.xlabel("Time Steps")
plt.ylabel("Temperature (°C)")
plt.show()


Exception: URL fetch failure on https://storage.googleapis.com/download.tensorflow.org/data/jena_climate_2009_2016.csv.zip: 403 -- Forbidden